Assume data cleaning works(preprocessing) is finished.
## **Task:**

1. built the training function
   * Accepts word2vec matrix and the rating corresponding to each row of matrix
2. bulid n-grams feature function
  * use nltk.ngrams(test_sentence,n)
3. then make a dictionary with n-grams
4. Word2vec 
5. built the classifier
6. Train the data set 
7. Calculate the accuracy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import nltk
from nltk import ngrams
from nltk import word_tokenize

import numpy as np
import pandas as pd

import tensorflow as tf
import numpy as np
import pprint
import re

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [3]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

#Load the data

In [4]:
path = '/content/drive/My Drive/cs182proj/NLP data set/preprocessed_remove_stopwords_stem.csv'
reviews = pd.read_csv(path)
reviews.head()

,stars,review
0,1.0,total bill horribl servic over gs crook actual...
1,5.0,ador travi hard rock new kelli cardena salon a...
2,5.0,say offic realli togeth organ friendli dr j ph...
3,5.0,went lunch steak sandwich delici caesar salad ...
4,1.0,today second out three session paid although f...


In [5]:
len(reviews)

533581

#Tokenize the text and remove Non-english text

In [0]:
reviews['review_tokenize'] = reviews['review'].apply(lambda r: word_tokenize(r) if(type(r)!=float) else 'NA')

In [0]:
reviews = reviews.iloc[list(reviews['review_tokenize'].apply(lambda r: r[0] != 'NA' and r[0] != 'unklang'))]

In [8]:
reviews.head()

,stars,review,review_tokenize
0,1.0,total bill horribl servic over gs crook actual...,"[total, bill, horribl, servic, over, gs, crook..."
1,5.0,ador travi hard rock new kelli cardena salon a...,"[ador, travi, hard, rock, new, kelli, cardena,..."
2,5.0,say offic realli togeth organ friendli dr j ph...,"[say, offic, realli, togeth, organ, friendli, ..."
3,5.0,went lunch steak sandwich delici caesar salad ...,"[went, lunch, steak, sandwich, delici, caesar,..."
4,1.0,today second out three session paid although f...,"[today, second, out, three, session, paid, alt..."


In [9]:
len(reviews)

529707

#Remove the low frequncy words

In [0]:
words_collection = []
review_tokenize = reviews['review_tokenize']
for i in review_tokenize:
  words_collection.extend(i)

In [11]:
len(words_collection)

28558674

In [0]:
words_collection = pd.Series(words_collection)

In [0]:
words_count = words_collection.value_counts()

In [14]:
len(words_count)

235598

In [60]:
low_freq_words = words_count[words_count.values <= 15000]
len(low_freq_words)

235237

In [61]:
sini_words = words_count[words_count.values > 15000]
len(sini_words)

361

In [0]:
reviews['sinificant words'] = reviews['review_tokenize'].apply(lambda r: [i for i in r if i in sini_words and len(i)>1])

##***Input of function train_NB_ngrams***

In [63]:
reviews.head(10)

,stars,review,review_tokenize,sinificant words
0,1.0,total bill horribl servic over gs crook actual...,"[total, bill, horribl, servic, over, gs, crook...","[total, bill, horribl, servic, over, actual, c..."
1,5.0,ador travi hard rock new kelli cardena salon a...,"[ador, travi, hard, rock, new, kelli, cardena,...","[hard, new, salon, alway, great, no, offer, se..."
2,5.0,say offic realli togeth organ friendli dr j ph...,"[say, offic, realli, togeth, organ, friendli, ...","[say, offic, realli, friendli, dr, great, veri..."
3,5.0,went lunch steak sandwich delici caesar salad ...,"[went, lunch, steak, sandwich, delici, caesar,...","[went, lunch, sandwich, delici, salad, absolut..."
4,1.0,today second out three session paid although f...,"[today, second, out, three, session, paid, alt...","[today, second, out, three, paid, first, went,..."
5,4.0,first admit not excit go la tavolta food snob ...,"[first, admit, not, excit, go, la, tavolta, fo...","[first, not, go, food, friend, go, dinner, loo..."
6,3.0,traci dessert big name hong kong one first mar...,"[traci, dessert, big, name, hong, kong, one, f...","[big, name, one, first, place, mani, year, cam..."
7,1.0,place gone down hill clearli cut back staff fo...,"[place, gone, down, hill, clearli, cut, back, ...","[place, down, cut, back, staff, food, qualiti,..."
8,2.0,realli look forward visit beer man war quickli...,"[realli, look, forward, visit, beer, man, war,...","[realli, look, visit, beer, favorit, good, swe..."
9,3.0,giant best buy regist not get big deal place,"[giant, best, buy, regist, not, get, big, deal...","[best, buy, not, get, big, deal, place]"


In [0]:
df = reviews[['stars','sinificant words']].sample(80000, replace=False)

#Training Function

In [0]:
# this clssifier works for different grams (n = 1,2,3...)

def trainNB(sub_reviews_vector, review_cate):
    """
    tsub_reviews_vector: a batch of the review text vectors, 
                         with shape: (dictionary_len, batch_size)
    review_cate: a list(np.array) of the classes (rating) of the reviews, 
                they look like [5.0, 1.0, 3.0, 4.0, 5.0, 2.0,.....]
    """
    """
    return: 1. a matrix in which each row represents the log of probabilities
            of features in the same class. 
            2. a dictionary with the ratings as the keys, and the
            probabilities as their values
    """

    batch_size = len(sub_reviews_vector)
    reviews_vec_len = len(sub_reviews_vector[0])
    
    #initialize features distribution for all classes, and set all entries 
    #equals to 1 (to avoid getting a 0 fraction), 5 represents the number of 
    #different classes
    features_dist = np.ones((5,reviews_vec_len)) 

    #initialize the sum of features for all classes, and set all entries 
    # equals to 1
    features_sum = np.ones(5)
    
    for i in range(len(sub_reviews_vector)):
      rating = review_cate[i]
      features_dist[int(rating)-1] += sub_reviews_vector[i]
      features_sum[int(rating)-1] += np.sum(sub_reviews_vector[i])
     
    #calcualte the log of probability of each features
    log_features_frac = np.ones((5,reviews_vec_len))
    for i in range(5):
      features_dist[i] /= features_sum[i]
      log_features_frac[i] = np.log(features_dist[i])

    #calculate the fraction of each class in the review category list
    class_proba = class_prob(review_cate)

    return [log_features_frac,class_proba]

#calculate the probability of each class 

In [0]:

def class_prob(review_cate):
    """
    review_cate: a list(np.array) of the class type of the reviews, 
                they look like [5.0, 1.0, 3.0, 4.0, 5.0, 2.0,.....]
    """  

    """
    return: a dictionary with the rating as the keys and the corresponding
            probability in the set review_cate
    """

    review_cate = pd.Series(review_cate)
    val_count = review_cate.value_counts()
    class_proba = {}
    size = len(review_cate)
    
    for i in range(len(val_count)):
      class_proba[val_count.keys()[i]] = val_count.values[i]/size 

    return class_proba

#Create n-gram feature word of the reviews

In [0]:
def create_ngrams_text(reviews, n):
  """
  reviews: a dataframe with the texts tokenize
  n: the number of grams
  """
  """
  return a list of reviews with n grams, and each n gram feature is 
  a tuple
  """
  ngrams_text = []
  for i in reviews:
    ngrams_review = ngrams(i,n)
    ngrams_review = [ng for ng in ngrams_review]
    ngrams_text.append(ngrams_review)

  return ngrams_text

# Built a dictionary with n-gram word


In [0]:
def create_ngram_dictionary(df, n):
  """
  reviews: a data frame of reviews text with different classes(rating)
          in the second column
  n: the number of grams
  """
  """
  return a list of non-repeated n grams vocabulary 
  """
  reviews = df.iloc[:,1]
  dictionary = set()
  for i in reviews:
    ngrams_review = ngrams(i,n)
    ngram_review = set(ng for ng in ngrams_review)
    dictionary = dictionary.union(ngram_review)
 
  dictionary = list(dictionary)

  return dictionary

#Word to vector

In [0]:
def setwords2vector(reviews, dictionary):
  """
  rewiews: a list of reviews in n-grams fromate
  dictionary: the dictionary(a set of unique words) of reviews text in n-grams fromate
  """
  """
  return a words to vector matrix with shape(reviews_size, dictionary_len)
  """
  num_row = len(reviews)
  num_column = len(dictionary)
  reviews2Matrix = np.zeros((len(reviews),len(dictionary)))
  for i in range(len(reviews)):
    for w in reviews[i]:
      if w in dictionary:
        reviews2Matrix[i][dictionary.index(w)] +=1
  return reviews2Matrix

#Get the text of reviews and the corresponding ratings separately

In [0]:
def get_reviews_rating(reviews):
  """
  reviews: the dataframe
  """
  """
  return a list of text of reviews and ratings 
  """
  revs = np.array(reviews.iloc[:,1])
  ratings = np.array(reviews.iloc[:,0])
  return revs, ratings

#Predict the rating

In [0]:
def NBclassifier(classifier, word2vector):
    """
    classifier: a list with element log_features_frac and class_proba
    log_features_frac: log value of the feature fraction with 
                       shape(5,dictionary_len), and the first row is the frac
                       of class 1.0 ... the last row is class 5.0
    class_proba: a dictionary with the class(rating) as key and there fraction 
                 in the data set as the value
    word2vector: a single vector representation of a review text with shape(1,dictionary_len)            
    """
    """
    return the predicted class(rating)
    """ 
    log_features_frac, class_proba = classifier
    proba_class = {}
    for i in class_proba:
      class_classifier = log_features_frac[int(i)-1]
      proba = np.log(class_proba[i]) + sum(class_classifier*word2vector)
      proba_class[proba] = i
    class_ = proba_class[max(proba_class)]
    return class_

#Calculate the accuracy

In [0]:
def get_accuracy(pred_rating, actual_rating):
    """
    pred_rating: a list of predicted rating
    actual_rating: a list of actual_rating with the same length as pred_rating
    """ 
    """
    return the accuracy of the prediction
    """ 
    size = len(pred_rating)
    match = [pred_rating[i]==actual_rating[i] for i in range(size)]
    accuracy = sum(match)/size
    return accuracy

In [0]:
def get__accuracy(pred_rating, actual_rating):
    """
    pred_rating: a list of predicted rating
    actual_rating: a list of actual_rating with the same length as pred_rating
    """ 
    """
    return the accuracy of the prediction
    """ 
    size = len(pred_rating)
    match = []
    for i in range(size):
      a = pred_rating[i]>=4.0 and actual_rating[i]>=4.0
      b = pred_rating[i]==3.0 and actual_rating[i]==3.0
      c = pred_rating[i]<=2.0 and actual_rating[i]<=2.0
      boo = a or b or c
      match.append(boo)      

    accuracy = sum(match)/size
    return accuracy

#Get training and test set

In [0]:
def get_train_test_set(fraction, df):
  """
  fraction: fraction of the test set
  df: data frame 
  """
  train, test = train_test_split(df, test_size=fraction)
  return train, test 

#Train Naive Bayes Classifier for n grams

In [0]:
def train_NB_ngrams(df, n):
    """
    dataSet: a dataframe with rating and the tokenized review text
    n: the number of grams
    """ 
    """
    return the accuracy of the n grams prediction 
    """ 
    #get training and test set
    training, test = get_train_test_set(0.2, df)

    #get reviews and rating of training and test set separately 
    tra_rev, tra_rating = get_reviews_rating(training)
    test_rev, test_rating = get_reviews_rating(test)

    #get n-grams train and test vocabulary
    tra_rev_ngrams = create_ngrams_text(tra_rev, n)
    test_rev_ngrams = create_ngrams_text(test_rev, n)

    #get n-grams dictionary
    dictionary = create_ngram_dictionary(df, n)

    #train and test vocabulary to vector
    tra_words2vec = setwords2vector(tra_rev_ngrams, dictionary)
    test_words2vec = setwords2vector(test_rev_ngrams, dictionary)

    #get classifier 
    classifier = trainNB(tra_words2vec, tra_rating)

    #predict rating
    preds = []
    for i in test_words2vec:
      prediction = NBclassifier(classifier, i)
      preds.append(prediction)
      
    #calculate the accuracy
    accu = get_accuracy(preds, test_rating)

    return [accu, preds, test_rating]

In [66]:
accuracy, pred_rating, actual_rating= train_NB_ngrams(df, 1)
print(str(1)+'-gram accuracy: ', accuracy)
print(str(1)+'-gram actual rating distribution: ', class_prob(actual_rating))
print(str(1)+'-gram predicted rating distribution: ', class_prob(pred_rating))

1-gram accuracy:  0.654125
1-gram actual rating distribution:  {5.0: 0.4939375, 1.0: 0.241875, 4.0: 0.133875, 2.0: 0.068125, 3.0: 0.0621875}
1-gram predicted rating distribution:  {5.0: 0.45725, 1.0: 0.2468125, 4.0: 0.1866875, 3.0: 0.06275, 2.0: 0.0465}


#1-gram: 

**# sample: 20000**

1-gram accuracy:  0.60575

1-gram actual rating distribution:  {5.0: 0.48275, 1.0: 0.233, 4.0: 0.14675, 3.0: 0.06875, 2.0: 0.06875}

1-gram predicted rating distribution: missing

**# sample: 30000**

(remove words appear less than 150)

1-gram accuracy:  0.6206666666666667

1-gram actual rating distribution:  {5.0: 0.482, 1.0: 0.24533333333333332, 4.0: 0.141, 2.0: 0.06966666666666667, 3.0: 0.062}

1-gram predicted rating distribution:  {5.0: 0.3835, 1.0: 0.22633333333333333, 4.0: 0.20133333333333334, 3.0: 0.11116666666666666, 2.0: 0.07766666666666666}

**# sample: 30000**

ccuracy:  0.6371666666666667

1-gram actual rating distribution:  {5.0: 0.505, 1.0: 0.23216666666666666, 4.0: 0.14266666666666666, 3.0: 0.06216666666666667, 2.0: 0.058}

1-gram predicted rating distribution:  {5.0: 0.40166666666666667, 1.0: 0.225, 4.0: 0.21316666666666667, 3.0: 0.08616666666666667, 2.0: 0.074}

**# sample: 30000**

(remove words appear less than 300)

1-gram accuracy:  0.6383333333333333

1-gram actual rating distribution:  {5.0: 0.4835, 1.0: 0.24583333333333332, 4.0: 0.1375, 2.0: 0.0685, 3.0: 0.06466666666666666}

1-gram predicted rating distribution:  {5.0: 0.38616666666666666, 1.0: 0.22583333333333333, 4.0: 0.20616666666666666, 3.0: 0.108, 2.0: 0.07383333333333333}

**# sample: 40000**

(remove words appear less than 500)

1-gram accuracy:  0.643125

1-gram actual rating distribution:  {5.0: 0.49175, 1.0: 0.246125, 4.0: 0.1335, 2.0: 0.065, 3.0: 0.063625}

1-gram predicted rating distribution:  {5.0: 0.40075, 1.0: 0.23025, 4.0: 0.2105, 3.0: 0.09, 2.0: 0.0685}


**# sample: 40000**

(remove words appear less than 900)

1-gram accuracy:  0.64825

1-gram actual rating distribution:  {5.0: 0.49525, 1.0: 0.237125, 4.0: 0.135625, 3.0: 0.068625, 2.0: 0.063375}

1-gram predicted rating distribution:  {5.0: 0.4035, 1.0: 0.22425, 4.0: 0.2135, 3.0: 0.089125, 2.0: 0.069625}

**# sample: 60000**

1-gram accuracy:  0.6444166666666666

(remove words appear less than 1200)

1-gram actual rating distribution:  {5.0: 0.49475, 1.0: 0.23941666666666667, 4.0: 0.13425, 2.0: 0.06816666666666667, 3.0: 0.06341666666666666}

1-gram predicted rating distribution:  {5.0: 0.40358333333333335, 1.0: 0.2325, 4.0: 0.21633333333333332, 3.0: 0.08091666666666666, 2.0: 0.06666666666666667}


**# sample: 60000**

(remove words appear less than 1000)

1-gram accuracy:  0.65325

1-gram actual rating distribution:  {5.0: 0.48483333333333334, 1.0: 0.24166666666666667, 4.0: 0.13958333333333334, 2.0: 0.07025, 3.0: 0.06366666666666666}

1-gram predicted rating distribution:  {5.0: 0.40066666666666667, 1.0: 0.23608333333333334, 4.0: 0.21825, 3.0: 0.08191666666666667, 2.0: 0.06308333333333334}

#Moving 500 features

**1-gram accuracy:  0.605 (# sample 10000)**

1-gram actual rating distribution:  {5.0: 0.4915, 1.0: 0.2445, 4.0: 0.134, 3.0: 0.0655, 2.0: 0.0645}

1-gram predicted rating distribution:  {5.0: 0.363, 1.0: 0.2205, 4.0: 0.204, 3.0: 0.1435, 2.0: 0.069}


**1-gram accuracy:  0.612 (# sample 20000)**

1-gram actual rating distribution:  {5.0: 0.5015, 1.0: 0.2485, 4.0: 0.13, 2.0: 0.0605, 3.0: 0.0595}

1-gram predicted rating distribution:  {5.0: 0.383, 1.0: 0.22, 4.0: 0.175, 3.0: 0.1375, 2.0: 0.0845}
<br>


**1-gram accuracy:  0.6348333333333334 (# sample 30000)**

1-gram actual rating distribution:  {5.0: 0.492, 1.0: 0.24, 4.0: 0.1335, 2.0: 0.0695, 3.0: 0.065}

1-gram predicted rating distribution:  {5.0: 0.3983333333333333, 1.0: 0.23016666666666666, 4.0: 0.203, 3.0: 0.10383333333333333, 2.0: 0.06466666666666666}
<br>


**1-gram accuracy:  0.634875 (40000)**

1-gram actual rating distribution:  {5.0: 0.488, 1.0: 0.24925, 4.0: 0.133375, 2.0: 0.06925, 3.0: 0.060125}

1-gram predicted rating distribution:  {5.0: 0.388, 1.0: 0.237, 4.0: 0.21025, 3.0: 0.099625, 2.0: 0.065125}

**1-gram accuracy:  0.647625(40000)**

1-gram actual rating distribution:  {5.0: 0.492625, 1.0: 0.24575, 4.0: 0.1325, 3.0: 0.066125, 2.0: 0.063}
1-gram predicted rating

distribution:  {5.0: 0.394125, 1.0: 0.2355, 4.0: 0.2135, 3.0: 0.083125, 2.0: 0.07375}


**1-gram accuracy:  0.655125 (40000)**

1-gram actual rating distribution:  {5.0: 0.49675, 1.0: 0.2435, 4.0: 0.13325, 2.0: 0.067, 3.0: 0.0595}

1-gram predicted rating distribution:  {5.0: 0.4055, 1.0: 0.232375, 4.0: 0.2115, 3.0: 0.08125, 2.0: 0.069375}

**1-gram accuracy:  0.636375 (40000)**

1-gram actual rating distribution:  {5.0: 0.491125, 1.0: 0.24375, 4.0: 0.13775, 2.0: 0.065, 3.0: 0.062375}

1-gram predicted rating distribution:  {5.0: 0.3945, 1.0: 0.231, 4.0: 0.219375, 3.0: 0.085375, 2.0: 0.06975}

#Removing 1000
**1-gram accuracy:  0.64225(40000)**

1-gram actual rating distribution:  {5.0: 0.4805, 1.0: 0.25025, 4.0: 0.1445, 3.0: 0.0635, 2.0: 0.06125}

1-gram predicted rating distribution:  {5.0: 0.394, 1.0: 0.239125, 4.0: 0.213375, 3.0: 0.0815, 2.0: 0.072}

**1-gram accuracy:  0.642(40000)**

1-gram actual rating distribution:  {5.0: 0.491125, 1.0: 0.2465, 4.0: 0.138125, 2.0: 0.064625, 3.0: 0.059625}

1-gram predicted rating distribution:  {5.0: 0.40175, 1.0: 0.231375, 4.0: 0.212125, 3.0: 0.085, 2.0: 0.06975}

**1-gram accuracy:  0.646125(40000)**

1-gram actual rating distribution:  {5.0: 0.481875, 1.0: 0.2535, 4.0: 0.136, 2.0: 0.06475, 3.0: 0.063875}

1-gram predicted rating distribution:  {5.0: 0.39, 1.0: 0.237625, 4.0: 0.215875, 3.0: 0.08375, 2.0: 0.07275}


**1-gram accuracy:  0.650625(80000)**

1-gram actual rating distribution:  {5.0: 0.4905, 1.0: 0.241875, 4.0: 0.133625, 2.0: 0.067125, 3.0: 0.066875}

1-gram predicted rating distribution:  {5.0: 0.4003125, 1.0: 0.2340625, 4.0: 0.2199375, 3.0: 0.0798125, 2.0: 0.065875}

# **Removing 750**
**1-gram accuracy:  0.6511666666666667(60000)**

1-gram actual rating distribution:  {5.0: 0.489, 1.0: 0.24091666666666667, 4.0: 0.13775, 2.0: 0.06708333333333333, 3.0: 0.06525}

1-gram predicted rating distribution:  {5.0: 0.4028333333333333, 1.0: 0.23758333333333334, 4.0: 0.21391666666666667, 3.0: 0.08775, 2.0: 0.057916666666666665}


**1-gram accuracy:  0.649 (60000)**

1-gram actual rating distribution:  {5.0: 0.48741666666666666, 1.0: 0.24791666666666667, 4.0: 0.13025, 2.0: 0.07025, 3.0: 0.06416666666666666}

1-gram predicted rating distribution:  {5.0: 0.399, 1.0: 0.242, 4.0: 0.21166666666666667, 3.0: 0.08291666666666667, 2.0: 0.06441666666666666}